In [1]:
import os
from tqdm import tqdm
import torchinfo
import torch
import torchvision
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt

In [2]:
TRAIN_INPUT_DIR = 'ISIC2018_Task1-2_Training_Input/'
TRAIN_GT_DIR = 'ISIC2018_Task1_Training_GroundTruth/'

VAL_INPUT_DIR = 'ISIC2018_Task1-2_Validation_Input/'
VAL_GT_DIR = 'ISIC2018_Task1_Validation_GroundTruth/'

BATCH_SIZE = 2
LEARNING_RATE = 1e-5
EPOCHS = 100
IM_H, IM_W = 384, 512

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
training_input_files = os.listdir(TRAIN_INPUT_DIR)
for filename in training_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    training_input_files.remove(filename)
n_train = len(training_input_files)

val_input_files = os.listdir(VAL_INPUT_DIR)
for filename in val_input_files:
  if (filename[-3:] != "jpg" and filename[-3:] != "png"):
    val_input_files.remove(filename)
n_val = len(val_input_files)
n_train -= n_train % BATCH_SIZE
n_val -= n_val % BATCH_SIZE
print(n_train, n_val)

2594 100


In [4]:
def centroid(img, lcc=False):
  if lcc:
    img = img.astype(np.uint8)
    nb_components, output, stats, centroids = cv.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    if len(sizes) > 2:
      max_label = 1
      max_size = sizes[1]

      for i in range(2, nb_components):
          if sizes[i] > max_size:
              max_label = i
              max_size = sizes[i]

      img2 = np.zeros(output.shape)
      img2[output == max_label] = 255
      img = img2

  if len(img.shape) > 2:
    M = cv.moments(img[:,:,1])
  else:
    M = cv.moments(img)

  if M["m00"] == 0:
    return (img.shape[0] // 2, img.shape[1] // 2)
  
  cX = int(M["m10"] / M["m00"])
  cY = int(M["m01"] / M["m00"])
  return (cX, cY)

def to_polar(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[0]/2.0)**2.0)+((input_img.shape[1]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS)
  return polar_image

def to_cart(input_img, center):
  input_img = input_img.astype(np.float32)
  value = np.sqrt(((input_img.shape[1]/2.0)**2.0)+((input_img.shape[0]/2.0)**2.0))
  polar_image = cv.linearPolar(input_img, center, value, cv.WARP_FILL_OUTLIERS + cv.WARP_INVERSE_MAP)
  polar_image = polar_image.astype(np.uint8)
  return polar_image

In [59]:
train_X_cart = torch.zeros((BATCH_SIZE, 3, IM_H, IM_W)).to(DEVICE)
train_y_cart = torch.zeros((BATCH_SIZE, 1, IM_H, IM_W)).to(DEVICE)
train_X_pol = torch.zeros((BATCH_SIZE, 3, IM_H, IM_W)).to(DEVICE)
train_y_pol = torch.zeros((BATCH_SIZE, 1, IM_H, IM_W)).to(DEVICE)
nb = n_train//BATCH_SIZE
with tqdm(total=n_train) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = training_input_files[i]
            in_image = cv.resize(cv.imread(TRAIN_INPUT_DIR + filename), (IM_W, IM_H))
            gt_image = cv.resize(cv.imread(TRAIN_GT_DIR + filename[:-4] + "_segmentation.png", cv.IMREAD_GRAYSCALE), (IM_W, IM_H))
            center = centroid(gt_image)
            gt_image = np.expand_dims(gt_image, axis=2)
            train_X_cart[i-l, :, :, :] = torch.tensor(in_image).transpose(2, 1).transpose(1, 0).to(DEVICE)
            train_y_cart[i-l, :, :, :] = torch.tensor(gt_image).transpose(2, 1).transpose(1, 0).to(DEVICE)
            
            in_image_pol = to_polar(in_image, center)
            gt_image_pol = to_polar(gt_image, center)
            gt_image_pol = np.expand_dims(gt_image_pol, axis=2)
            train_X_pol[i-l, :, :, :] = torch.tensor(in_image_pol).transpose(2, 1).transpose(1, 0).to(DEVICE)
            train_y_pol[i-l, :, :, :] = torch.tensor(gt_image_pol).transpose(2, 1).transpose(1, 0).to(DEVICE)
            
            pbar.update(1)

        train_y_cart = torch.div(train_y_cart, 128, rounding_mode='trunc')
        train_y_pol = torch.div(train_y_pol, 128, rounding_mode='trunc')
        
        torch.save(train_X_cart, f"./Preproc/train_X_cart_{f}.torch")
        torch.save(train_y_cart, f"./Preproc/train_y_cart_{f}.torch")
        torch.save(train_X_pol, f"./Preproc/train_X_pol_{f}.torch")
        torch.save(train_y_pol, f"./Preproc/train_y_pol_{f}.torch")
        
   
val_X_cart = torch.zeros((BATCH_SIZE, 3, IM_H, IM_W)).to(DEVICE)
val_y_cart = torch.zeros((BATCH_SIZE, 1, IM_H, IM_W)).to(DEVICE)
val_X_pol = torch.zeros((BATCH_SIZE, 3, IM_H, IM_W)).to(DEVICE)
val_y_pol = torch.zeros((BATCH_SIZE, 1, IM_H, IM_W)).to(DEVICE)
nb = n_val//BATCH_SIZE
with tqdm(total=n_val) as pbar:
    for f in range(nb):
        l, r = f*BATCH_SIZE, (f+1)*BATCH_SIZE
        filename = None
        for i in range(l, r):
            filename = val_input_files[i]
            in_image = cv.resize(cv.imread(VAL_INPUT_DIR + filename), (IM_W, IM_H))
            gt_image = cv.resize(cv.imread(VAL_GT_DIR + filename[:-4] + "_segmentation.png", cv.IMREAD_GRAYSCALE), (IM_W, IM_H))
            center = centroid(gt_image)
            gt_image = np.expand_dims(gt_image, axis=2)
            val_X_cart[i-l, :, :, :] = torch.tensor(in_image).transpose(2, 1).transpose(1, 0).to(DEVICE)
            val_y_cart[i-l, :, :, :] = torch.tensor(gt_image).transpose(2, 1).transpose(1, 0).to(DEVICE)
            
            in_image_pol = to_polar(in_image, center)
            gt_image_pol = to_polar(gt_image, center)
            gt_image_pol = np.expand_dims(gt_image_pol, axis=2)
            val_X_pol[i-l, :, :, :] = torch.tensor(in_image_pol).transpose(2, 1).transpose(1, 0).to(DEVICE)
            val_y_pol[i-l, :, :, :] = torch.tensor(gt_image_pol).transpose(2, 1).transpose(1, 0).to(DEVICE)
            
            pbar.update(1)

        val_y_cart = torch.div(val_y_cart, 128, rounding_mode='trunc')
        val_y_pol = torch.div(val_y_pol, 128, rounding_mode='trunc')
        
        torch.save(val_X_cart, f"./Preproc/val_X_cart_{f}.torch")
        torch.save(val_y_cart, f"./Preproc/val_y_cart_{f}.torch")
        torch.save(val_X_pol, f"./Preproc/val_X_pol_{f}.torch")
        torch.save(val_y_pol, f"./Preproc/val_y_pol_{f}.torch")
        
   
   

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.40it/s]


In [5]:
def train(model):
  num_batches = n_train // BATCH_SIZE
  model.train()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    for batch in range(num_batches):
      X = torch.load(f"./Preproc/train_X_pol_{batch}.torch")
      y = torch.load(f"./Preproc/train_y_pol_{batch}.torch")
      loss, met = model.fit(X, y)
      t_loss += loss*len(X)
      t_met += met
      proc += len(X)
      pbar.update(1)
      pbar.set_description(f"Avg. Loss: {t_loss/proc: .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_train, t_met/n_train)

def test(model):
  num_batches = n_val // BATCH_SIZE
  model.eval()
  t_loss, t_met, proc = 0, torch.tensor([0., 0., 0., 0.]), 0
  with tqdm(total=num_batches) as pbar:
    pbar.set_description("Avg.Loss: 0.0000, Avg. Accuracy: 0.0000")
    with torch.no_grad():
      for batch in range(num_batches):
        X = torch.load(f"./Preproc/val_X_pol_{batch}.torch")
        y = torch.load(f"./Preproc/val_y_pol_{batch}.torch")
        loss, met = model.test(X, y)
        t_loss += loss*len(X)
        t_met += met
        proc += len(X)
        pbar.update(1)
        pbar.set_description(f"Avg. Loss: {t_loss/proc : .4f}, Avg. Accuracy: {(t_met[0] + t_met[3])/proc: .4f}")
  return (t_loss/n_val, t_met/n_val)

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from LossNormModel import LossNormModel

class UNetRes50(LossNormModel):
    def __init__(self, learning_rate=None,  loss_fn=None, optimizer=None, device=None, weight_decay=None):
        super(UNetRes50, self).__init__()

        if (device is None):
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        if (learning_rate is None):
            self.learning_rate = 1e-5
        else:
            self.learning_rate = learning_rate
        
        if (weight_decay is None):
            self.weight_decay = 1e-5
        else:
            self.weight_decay = weight_decay

        resnet50 = torchvision.models.resnet50(weights="DEFAULT")
        self.e1 = nn.Sequential(resnet50._modules['conv1'], resnet50._modules['bn1'], resnet50._modules['relu'])
        self.e2 = nn.Sequential(resnet50._modules['maxpool'], resnet50._modules['layer1'])
        self.e3 = nn.Sequential(resnet50._modules['layer2'])
        self.e4 = nn.Sequential(resnet50._modules['layer3'])
        
        self.c1 = self.convblock(512+128, 128)
        self.c2 = self.convblock(256+64, 64)
        self.c3 = self.convblock(64+32, 32)
        self.c4 = self.convblock(3+16, 16)
        
        self.u1 = nn.ConvTranspose2d(1024, 128, (2, 2), stride=(2, 2))
        self.u2 = nn.ConvTranspose2d(128, 64, (2, 2), stride=(2, 2))
        self.u3 = nn.ConvTranspose2d(64, 32, (2, 2), stride=(2, 2))
        self.u4 = nn.ConvTranspose2d(32, 16, (2, 2), stride=(2, 2))
        
        self.out = nn.Conv2d(16, 1, (1, 1), padding="same")

        if (loss_fn is None):
            self.loss_fn = nn.BCELoss()
        else:
            self.loss_fn = loss_fn

        if (optimizer is None):
            self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        else:
            self.optimizer = optimizer(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)


        self = self.to(self.device)

    def setfinetuning(self, option):
        for param in self.e1.parameters():
            param.requires_grad_(option)
            
        for param in self.e2.parameters():
            param.requires_grad_(option)
            
        for param in self.e3.parameters():
            param.requires_grad_(option)
            
        for param in self.e4.parameters():
            param.requires_grad_(option)

    def convblock(self, in_features, out_features):
        block = nn.Sequential(
            nn.Conv2d(in_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU(),
            nn.Conv2d(out_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU()
            )
        block = block.to(self.device)
        return block


    def forward(self, X):
        X = X.to(self.device)

        s1 = self.e1(X)
        s2 = self.e2(s1)
        s3 = self.e3(s2)
        s4 = self.e4(s3)
        
        s4 = self.u1(s4)
        s3 = self.c1(torch.cat([s3, s4], dim=1))
        s4 = None

        s3 = self.u2(s3)
        s2 = self.c2(torch.cat([s2, s3], dim=1))
        s3 = None

        s2 = self.u3(s2)
        s1 = self.c3(torch.cat([s1, s2], dim=1))
        s2 = None

        s1 = self.u4(s1)
        X = self.c4(torch.cat([X, s1], dim=1))
        s1 = None

        X = torch.sigmoid(self.out(X))

        return X


In [51]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from LossNormModel import LossNormModel

class UNetRes50Spat(LossNormModel):
    def __init__(self, learning_rate=None,  loss_fn=None, optimizer=None, device=None, weight_decay=None):
        super(UNetRes50Spat, self).__init__()

        if (device is None):
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
        else:
            self.device = device

        if (learning_rate is None):
            self.learning_rate = 1e-5
        else:
            self.learning_rate = learning_rate
        
        if (weight_decay is None):
            self.weight_decay = 1e-5
        else:
            self.weight_decay = weight_decay

        resnet50 = torchvision.models.resnet50(weights="DEFAULT")
        self.e1 = nn.Sequential(resnet50._modules['conv1'], resnet50._modules['bn1'], resnet50._modules['relu'])
        self.e2 = nn.Sequential(resnet50._modules['maxpool'], resnet50._modules['layer1'])
        self.e3 = nn.Sequential(resnet50._modules['layer2'])
        self.e4 = nn.Sequential(resnet50._modules['layer3'])
        
        self.c1 = self.convblock(512+128+2, 128)
        self.c2 = self.convblock(256+64+2, 64)
        self.c3 = self.convblock(64+32+2, 32)
        self.c4 = self.convblock(3+16+2, 16)
        
        self.u1 = nn.ConvTranspose2d(1024+2, 128, (2, 2), stride=(2, 2))
        self.u2 = nn.ConvTranspose2d(128, 64, (2, 2), stride=(2, 2))
        self.u3 = nn.ConvTranspose2d(64, 32, (2, 2), stride=(2, 2))
        self.u4 = nn.ConvTranspose2d(32, 16, (2, 2), stride=(2, 2))
        
        self.out = nn.Conv2d(16, 1, (1, 1), padding="same")

        if (loss_fn is None):
            self.loss_fn = nn.BCELoss()
        else:
            self.loss_fn = loss_fn

        if (optimizer is None):
            self.optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        else:
            self.optimizer = optimizer(self.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)


        self = self.to(self.device)

    def setfinetuning(self, option):
        for param in self.e1.parameters():
            param.requires_grad_(option)
            
        for param in self.e2.parameters():
            param.requires_grad_(option)
            
        for param in self.e3.parameters():
            param.requires_grad_(option)
            
        for param in self.e4.parameters():
            param.requires_grad_(option)

    def convblock(self, in_features, out_features):
        block = nn.Sequential(
            nn.Conv2d(in_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU(),
            nn.Conv2d(out_features, out_features, (3, 3), padding="same"),
            nn.BatchNorm2d(out_features),
            nn.ReLU()
            )
        block = block.to(self.device)
        return block


    def forward(self, X):
        X = X.to(self.device)
        
        spat = torch.ones((X.shape[0], 1, X.shape[2], X.shape[3])).to(self.device)
        i_spat = (torch.cumsum(spat, dim=2) - 1)
        j_spat = (torch.cumsum(spat, dim=3) - 1)

        s1 = self.e1(X)
        s2 = self.e2(s1)
        s3 = self.e3(s2)
        s4 = self.e4(s3)
        s4 = torch.cat([s4, i_spat[:, :, :s4.shape[2], :s4.shape[3]] / s4.shape[2], j_spat[:, :, :s4.shape[2], :s4.shape[3]] / s4.shape[3]], dim=1)
        
        s4 = self.u1(s4)
        s3 = torch.cat([s3, s4, i_spat[:, :, :s3.shape[2], :s3.shape[3]] / s3.shape[2], j_spat[:, :, :s3.shape[2], :s3.shape[3]] / s3.shape[2]], dim=1)
        s3 = self.c1(s3)

        s3 = self.u2(s3)
        s2 = torch.cat([s2, s3, i_spat[:, :, :s2.shape[2], :s2.shape[3]] / s2.shape[2], j_spat[:, :, :s2.shape[2], :s2.shape[3]] / s2.shape[2]], dim=1)
        s2 = self.c2(s2)

        s2 = self.u3(s2)
        s1 = torch.cat([s1, s2, i_spat[:, :, :s1.shape[2], :s1.shape[3]] / s1.shape[2], j_spat[:, :, :s1.shape[2], :s1.shape[3]] / s1.shape[2]], dim=1)
        s1 = self.c3(s1)

        s1 = self.u4(s1)
        X = torch.cat([X, s1, i_spat / X.shape[2], j_spat / X.shape[3]], dim=1)
        X = self.c4(X)

        X = torch.sigmoid(self.out(X))

        return X


In [52]:
net = UNetRes50Spat(learning_rate=5e-6, weight_decay=1.2e-2)

In [35]:
torchinfo.summary(net, input_size=(BATCH_SIZE, 3, IM_W, IM_H))

Layer (type:depth-idx)                        Output Shape              Param #
UNetRes50Spat                                 [2, 1, 512, 384]          --
├─Sequential: 1-1                             [2, 64, 256, 192]         --
│    └─Conv2d: 2-1                            [2, 64, 256, 192]         9,408
│    └─BatchNorm2d: 2-2                       [2, 64, 256, 192]         128
│    └─ReLU: 2-3                              [2, 64, 256, 192]         --
├─Sequential: 1-2                             [2, 256, 128, 96]         --
│    └─MaxPool2d: 2-4                         [2, 64, 128, 96]          --
│    └─Sequential: 2-5                        [2, 256, 128, 96]         --
│    │    └─Bottleneck: 3-1                   [2, 256, 128, 96]         75,008
│    │    └─Bottleneck: 3-2                   [2, 256, 128, 96]         70,400
│    │    └─Bottleneck: 3-3                   [2, 256, 128, 96]         70,400
├─Sequential: 1-3                             [2, 512, 64, 48]          --
│   

In [53]:
net.setfinetuning(False)
for t in range(EPOCHS):
    torch.cuda.empty_cache()
    print(f"Epoch {t+1} ---------------------")
    print(f"Training Set -----")
    train_loss, _ = train(net)
    print(f"Validation Set -----")
    train_loss, met = test(net)
    print(f"\rIoU: {met[0] / (met[0] + met[1] + met[2]): .4f}, Dice: {met[0] / (met[0] + 0.5 * (met[1] + met[2])): .4f}, Acc: {met[0] + met[3] : .4f}")

Epoch 1 ---------------------
Training Set -----


Avg. Loss:  55.2467, Avg. Accuracy:  0.6787: 100%|█████████████████████████████████| 1297/1297 [07:29<00:00,  2.88it/s]


Validation Set -----


Avg. Loss:  67.2501, Avg. Accuracy:  0.7837: 100%|█████████████████████████████████████| 50/50 [00:11<00:00,  4.33it/s]


IoU:  0.5983, Dice:  0.7487, Acc:  0.7837
Epoch 2 ---------------------
Training Set -----


...

Validation Set -----


Avg. Loss:  66.7681, Avg. Accuracy:  0.9180: 100%|█████████████████████████████████████| 50/50 [00:11<00:00,  4.35it/s]


IoU:  0.7918, Dice:  0.8838, Acc:  0.9180
Epoch 91 ---------------------
Training Set -----


Avg. Loss:  55.2825, Avg. Accuracy:  0.9908:  93%|██████████████████████████████▊  | 1211/1297 [06:59<00:29,  2.89it/s]


KeyboardInterrupt: 